In [14]:
import pandas as pd
import plotly.express as px

In [46]:
def extract_airline_data(airline_name):
    dict_airline = {}
    for i in range(2010, 2023):
        dict_airline[i] = {}
        file_name = f'resumo_anual_{i}.csv'
        df = pd.read_csv(file_name, sep=';', encoding='ISO-8859-1')
        df = df[['EMPRESA (SIGLA)', 'MÊS', 'NATUREZA', 'PASSAGEIROS PAGOS', 'ASSENTOS']]
        df_airline = df[df['EMPRESA (SIGLA)'] == airline_name]
        df_airline = df_airline.dropna()

        # Assentos Totais
        assentos_totais = int(df_airline['ASSENTOS'].sum())
        dict_airline[i]['assentos_totais'] = assentos_totais

        # Passageiros Pagos
        passageiros_pgs = int(df_airline['PASSAGEIROS PAGOS'].sum())
        dict_airline[i]['passageiros_pgs'] = passageiros_pgs

        # Passageiros Totais Doméstica
        domestica = df_airline[df_airline['NATUREZA'] == 'DOMÉSTICA']
        assentos_domestica = int(domestica['ASSENTOS'].sum())
        dict_airline[i]['assentos_domestica'] = assentos_domestica

        # Passageiros Pagos Doméstica
        passageiros_pgs_domestica = int(domestica['PASSAGEIROS PAGOS'].sum())
        dict_airline[i]['passageiros_pgs_domestica'] = passageiros_pgs_domestica

        # Assentos doméstica desocupados
        assentos_domestica_desocupados = assentos_domestica - passageiros_pgs_domestica

        # Vacância doméstica
        if assentos_domestica != 0:
            vacancia_domestica = (assentos_domestica_desocupados / assentos_domestica) * 100
        else:
            vacancia_domestica = 0
        dict_airline[i]['vacancia_domestica'] = vacancia_domestica

        # Passageiros Totais Internacional
        internacional = df_airline[df_airline['NATUREZA'] == 'INTERNACIONAL']
        assentos_internacional = int(internacional['ASSENTOS'].sum())
        dict_airline[i]['assentos_internacional'] = assentos_internacional

        # Passageiros Pagos Internacional
        passageiros_pgs_internacional = int(internacional['PASSAGEIROS PAGOS'].sum())
        dict_airline[i]['passageiros_pgs_internacional'] = passageiros_pgs_internacional

        # Assentos desocupados Internacional
        assentos_internacional_desocupados = assentos_internacional - passageiros_pgs_internacional

        # Vacância Internacional
        if assentos_internacional != 0:
            vacancia_internacional = (assentos_internacional_desocupados / assentos_internacional) * 100
        else:
            vacancia_internacional = 0
        dict_airline[i]['vacancia_internacional'] = vacancia_internacional
        
    df = pd.DataFrame(dict_airline).T
    df_print = df[['assentos_totais', 'passageiros_pgs', 'assentos_domestica',
               'passageiros_pgs_domestica',
               'assentos_internacional', 'passageiros_pgs_internacional']]
    df_print.reset_index(inplace=True)

    fig = px.bar(df_print, x='index', y=['assentos_totais', 'passageiros_pgs','assentos_domestica','passageiros_pgs_domestica','assentos_internacional', 'passageiros_pgs_internacional'], 
                 barmode='group', title='Estatísticas de transporte aéreo no Brasil')
    fig.update_layout(
    xaxis_title='Ano',
    yaxis_title='Passageiros',
    title={
    'text': f'Estatísticas de transporte aéreo da {airline_name}',
    'y':0.95,
    'x':0.5,
    'xanchor': 'center',
    'yanchor': 'top'})
    fig.show()        


    return df


In [48]:
extract_airline_data('GLO', 'GOL')

C:\Users\Pichau\AppData\Local\Temp\ipykernel_9848\4117431075.py:6: DtypeWarning:

Columns (24,25,29) have mixed types. Specify dtype option on import or set low_memory=False.



,assentos_totais,passageiros_pgs,assentos_domestica,passageiros_pgs_domestica,vacancia_domestica,assentos_internacional,passageiros_pgs_internacional,vacancia_internacional
2010,49281882.0,27945339.0,46602796.0,26580359.0,42.964025,2679086.0,1364980.0,49.050534
2011,52784362.0,31065385.0,50172167.0,29823733.0,40.557216,2612195.0,1241652.0,52.467101
2012,51628338.0,30946996.0,48980818.0,29661986.0,39.441628,2647520.0,1285010.0,51.463634
2013,54618916.0,33144263.0,51768378.0,31680004.0,38.804333,2850538.0,1464259.0,48.632188
2014,53480784.0,36461226.0,50510989.0,34669981.0,31.361508,2969795.0,1791245.0,39.684557
2015,52432303.0,36050794.0,49489674.0,34162320.0,30.970812,2942629.0,1888474.0,35.823578
2016,43561838.0,31846286.0,40998605.0,29985494.0,26.862160,2563233.0,1860792.0,27.404493
2017,41862267.0,32099327.0,39369963.0,30223330.0,23.232516,2492304.0,1875997.0,24.728404
2018,43003131.0,33253771.0,40490985.0,31419184.0,22.404496,2512146.0,1834587.0,26.971323
2019,45829320.0,36037764.0,42893680.0,34104893.0,20.489702,2935640.0,1932871.0,34.158446


In [49]:
extract_airline_data('TAM', 'TAM')

C:\Users\Pichau\AppData\Local\Temp\ipykernel_9848\4117431075.py:6: DtypeWarning:

Columns (24,25,29) have mixed types. Specify dtype option on import or set low_memory=False.



,assentos_totais,passageiros_pgs,assentos_domestica,passageiros_pgs_domestica,vacancia_domestica,assentos_internacional,passageiros_pgs_internacional,vacancia_internacional
2010,50621692.0,30771326.0,45969275.0,27177492.0,40.879007,4652417.0,3593834.0,22.753399
2011,52472623.0,32360811.0,47433183.0,28356272.0,40.218492,5039440.0,4004539.0,20.536032
2012,51208640.0,34691612.0,46016227.0,30528478.0,33.657147,5192413.0,4163134.0,19.822749
2013,48378703.0,35286205.0,42832344.0,30837788.0,28.003501,5546359.0,4448417.0,19.795725
2014,47534852.0,36081818.0,42191364.0,31641648.0,25.004444,5343488.0,4440170.0,16.905025
2015,46817661.0,36121758.0,40865019.0,31234937.0,23.565588,5952642.0,4886821.0,17.905008
2016,42629017.0,33698271.0,36464715.0,28587369.0,21.602653,6164302.0,5110902.0,17.088715
2017,41697555.0,33479026.0,35227570.0,28021679.0,20.455260,6469985.0,5457347.0,15.651319
2018,42774519.0,34082797.0,35958365.0,28466249.0,20.835530,6816154.0,5616548.0,17.599456
2019,45450597.0,37035805.0,38876916.0,31488794.0,19.003879,6573681.0,5547011.0,15.617886


In [50]:
extract_airline_data('AZU','AZUL')

C:\Users\Pichau\AppData\Local\Temp\ipykernel_9848\4117431075.py:6: DtypeWarning:

Columns (24,25,29) have mixed types. Specify dtype option on import or set low_memory=False.



,assentos_totais,passageiros_pgs,assentos_domestica,passageiros_pgs_domestica,vacancia_domestica,assentos_internacional,passageiros_pgs_internacional,vacancia_internacional
2010,5254416.0,4061894.0,5250758.0,4061243.0,22.654158,3658.0,651.0,82.203390
2011,10016072.0,7269969.0,10015836.0,7269873.0,27.416214,236.0,96.0,59.322034
2012,13926130.0,9815219.0,13926130.0,9815219.0,29.519407,0.0,0.0,0.000000
2013,18377802.0,12886397.0,18377684.0,12886279.0,29.880833,118.0,118.0,0.000000
2014,27173184.0,19384797.0,27148081.0,19363614.0,28.674097,25103.0,21183.0,15.615663
2015,28216564.0,19902655.0,27741809.0,19505493.0,29.689181,474755.0,397162.0,16.343798
2016,26958312.0,19528491.0,26394384.0,19044126.0,27.847810,563928.0,484365.0,14.108716
2017,27949734.0,21141455.0,27047807.0,20347145.0,24.773402,901927.0,794310.0,11.931897
2018,29164244.0,22179223.0,27725723.0,20941567.0,24.468815,1438521.0,1237656.0,13.963300
2019,33857732.0,26574182.0,32281169.0,25225218.0,21.857793,1576563.0,1348964.0,14.436404
